In [65]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2
from datetime import datetime
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv
import plotly.express as px
from Kafka import KafkaReader

load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [98]:
class InfoMap:
    df_map_data = None
    df_locations = None
    kafka = KafkaReader('weather.cleaned')

    def __init__(self) -> None:
        self.df_locations = self.load_locations()
        token = os.getenv('MAPBOXKEY')
        px.set_mapbox_access_token(token)

    def close_kafka(self) -> None:
        self.kafka.consumer.close()
        
    def get_forecasts(self) -> pd.DataFrame:
        df = pd.DataFrame(columns=['datetime','temp','city'])
        messages = self.kafka.retrieve()
        if messages:
            maxKey = pd.to_datetime(pd.DataFrame(messages.keys())[0]).max().strftime('%d.%m.%Y %H:%M')
            for f in messages[maxKey]['cities']:
                    city = f['city']
                    for w in f['weather']:
                        dt = w['dt']
                        temp = w['temp']
                        df.loc[len(df), df.columns] = dt, temp, city
        return df

    def load_locations(self) -> pd.DataFrame:
        df = pd.read_json('locations.json').T
        return df

    def get_map_data(self) -> None:
        forecasts = self.get_forecasts()
        df = pd.merge(left=forecasts, right=self.df_locations, left_on='city', right_index=True)
        df['temp'] = pd.to_numeric(df['temp'])
        #df['datetime'] = pd.to_datetime(df['datetime'])
        if not df.empty:
            self.df_map_data = df
    
    def plot_map(self) -> None:
        if not self.df_map_data.empty:
            fig = px.scatter_mapbox(self.df_map_data, lat="latitude", lon="longitude", text="city", color="temp", width=1440, height=800,
                            color_continuous_scale=px.colors.cyclical.IceFire, range_color=(-10, 40), animation_frame='datetime', animation_group='city', custom_data=['temp'])
            fig.update_traces(hovertemplate =
                                "<b>%{text} </b><br>" +
                                "Temperature: %{customdata[0]} °C<br>" +
                                "Hinweise:")
            fig.update_traces(marker={'size': 20, 'opacity': 0.75})
            fig.update_coloraxes(colorbar_title_text="Temperature", colorbar_title_side="top", colorbar_ypad=20,colorbar_tickformat=".1f", colorbar_ticksuffix=" °C")
            fig['layout']['margin']=dict(r=0, t=0, l=0, b=0)
            fig['layout']['sliders'][0]['currentvalue']={"prefix": "Date and time: "}
            fig['layout']['sliders'][0]['pad']=dict(r=0, t=10, l=0, b=20)
            fig["layout"].pop("updatemenus")
            fig.show()
        else:
            print("No information available to plot yet!")
    
    def update_map(self) -> None:
        self.get_map_data()
        self.plot_map()

infoMap = InfoMap()

In [99]:
infoMap.get_map_data()
infoMap.plot_map()
infoMap.update_map()

AttributeError: 'Figure' object has no attribute '_get_validator'